In [1]:
class BraidSignGroup:
    def __init__(self, n, p, l):
        self.p = p
        self.l = l
        self.n = n
        self.B = BraidGroup(n)
        self.identityElem = self.B([])
        self.gens = self.B.generators()
        self.leftGens = [g for g in self.gens if self.gens.index(g) + 1 <= (int(n / 2) - 1)]
        self.rightGens = [g for g in self.gens if self.gens.index(g) + 1 >= (int(n / 2) + 1)]
        self.magic = 5


    def hash(self, U, m, T):
        parts = [U]
        parts.append(int("".join([str(part) for part in [elem.encode("utf-8").hex() for elem in m]]), 16))
        #parts.append(int("".join([str(part) for part in [elem.encode("hex") for elem in m]]), 16))
        parts.append(self.getIntFromBraid(T))
        return int("".join([str(part) for part in parts])) % (self.p-1) + 1
    
    def hash1(self, m):
        return self.B([int(d) % (self.B.strands() - 1) + 1 for d in str(abs(hash(m)))])

    def getPowForLeftForm(self, leftForm):
        deltaPower = leftForm[0]
        if deltaPower == self.identityElem:
            return 0
        else:
            return deltaPower.exponent_sum()/self.B.delta().exponent_sum()


    def getIntFromBraid(self, braid):
        leftForm = braid.left_normal_form()
        parts = [self.getPowForLeftForm(leftForm)]
        for i in range(1, len(leftForm)):
            parts.append(int("".join([str(digit) for digit in leftForm[i].permutation()])))
        return int("".join([str(part) for part in parts]))


    def ifFromSubgroupL(self, braid):
        leftForm = braid.left_normal_form()
        inf = self.getPowForLeftForm(leftForm)
        sup = inf + len(leftForm) - 1
        if(0 <= inf and inf <= sup and sup <= self.l):
            return True
        return False

    def randLBElement(self):
        r = random.randint(1, self.magic)
        e = self.identityElem
        for i in range(0, r):
            e *= random.choice(self.leftGens)
        return e

    def randRBElement(self):
        r = random.randint(1, self.magic)
        e = self.identityElem
        for i in range(0, r):
            e *= random.choice(self.rightGens)
        return e
    def randLBlElement(self):
        while True:
            elem = self.randLBElement()
            if(self.ifFromSubgroupL(elem)):
                return elem


    def randRBlElement(self):
        while True:
            elem = self.randRBElement()
            if(self.ifFromSubgroupL(elem)):
                return elem

# INIT

In [2]:
bsg = BraidSignGroup(9, 17, 5)

In [25]:
emails = [
'phish@att.net',
'gumpish@aol.com',
'fatelk@comcast.net',
'wkrebs@verizon.net',
'kmself@aol.com',
'sthomas@yahoo.com',
'lishoy@gmail.com',
'muzzy@verizon.net',
'cameron@mac.com',
'neonatus@live.com',
'jramio@verizon.net',
'msusa@mac.com',
]
len(emails)

12

In [26]:
n = 12
t = 8

In [27]:
MK = bsg.randLBElement() # master key

In [28]:
S = randint(1, 97)
S

70

In [44]:
p = 425776014743526829624250629399
R = PolynomialRing(GF(p), 'x')
R, x = R.objgen()

In [45]:
f = R.random_element(t-1)
f = f - f % x + S
f

24224576952094378612081184921*x^7 + 353558348783471185147939897874*x^6 + 155102669868851680222693629212*x^5 + 44556402663319110725102162567*x^4 + 405648469560589116810745130442*x^3 + 322166312392659793964639384151*x^2 + 181423861179578808196669446864*x + 70

In [46]:
X = [hash(e) % p if hash(e) % p != 0 else 1 for e in emails]
Y = [f(x) for x in X]
X, Y

([5089000117413736082,
  3829310647238012714,
  1264183756410784295,
  425776014739629859158391272903,
  425776014737713878572803927365,
  425776014736781648236069372993,
  7828386037061962381,
  6998301396780504852,
  8191655294231708130,
  1655757601632062722,
  4807671426118815052,
  5806903836064515676],
 [104116276807776751071005850640,
  91893051024319818695122853693,
  315743277885918124882483999506,
  398427178028449392361861166739,
  213004332783286242143334272907,
  62879055574988279895497291090,
  381579754641945698239521807422,
  274842569547182019443387539088,
  12762328634704455236374143872,
  177833959185348940283413523477,
  244884724573558152843046860479,
  233498733291703896113191173169])

# SIGN

In [47]:
import random 
t_ids = list(range(len(emails)))
random.shuffle(t_ids)
t_ids = t_ids[:t]

In [48]:
t_X = [X[i] for i in t_ids]
t_Y = [Y[i] for i in t_ids]
points = list(zip(t_X, t_Y))
points

[(7828386037061962381, 381579754641945698239521807422),
 (1655757601632062722, 177833959185348940283413523477),
 (3829310647238012714, 91893051024319818695122853693),
 (5806903836064515676, 233498733291703896113191173169),
 (425776014737713878572803927365, 213004332783286242143334272907),
 (4807671426118815052, 244884724573558152843046860479),
 (425776014739629859158391272903, 398427178028449392361861166739),
 (425776014736781648236069372993, 62879055574988279895497291090)]

In [49]:
f = R.lagrange_polynomial(points)
f

24224576952094378612081184921*x^7 + 353558348783471185147939897874*x^6 + 155102669868851680222693629212*x^5 + 44556402663319110725102162567*x^4 + 405648469560589116810745130442*x^3 + 322166312392659793964639384151*x^2 + 181423861179578808196669446864*x + 70

In [229]:
S = f%x
S

461

In [231]:
sign_k = MK**S
sign_k

(s2^2*s0)^461

In [232]:
msg = 'hello'
h = bsg.hash1(msg)

In [233]:
sign = sign_k * h / sign_k

# VERIFICATION

In [235]:
sign.is_conjugated(bsg.hash1(msg))

True

84342368487090800366523834928142263660104883695016514377462985829716817089965

In [55]:
a = [4514920055187772626, 310336322056855043, 3255553097392970539, 8365634444100954542, 5012798650383826122, 1666657136041441915, 425776014737176155882656980968, 425776014739939764057783600870]
sorted(a)

[310336322056855043,
 1666657136041441915,
 3255553097392970539,
 4514920055187772626,
 5012798650383826122,
 8365634444100954542,
 425776014737176155882656980968,
 425776014739939764057783600870]